In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# .env 파일에 LANGCHAIN_API_KEY를 입력합니다.
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

# LangSmith 추적을 원하지 않을 경우
# logging.langsmith("CH01-Basic", set_enable=False)

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


### 데이터를 효과적으로 전달하는 방법
- `RunnalbePassthrough`는 입력을 변경하지 않거나 추가 키를 더하여 전달할 수 있습니다.
- `RunnablePassthrough()`가 단독으로 호출되면, 단순히 입력을 받아 그대로 전달합니다.
- `RunnablePassthrough.assign(...)`방식으로 호출되면, 입력을 받아 assign 함수에 전달된 추가 인수를 추가합니다.

### RunnablePassthrough

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# prompt와 llm을 생성합니다.
prompt = PromptTemplate.from_template("{num}의 10배는?")
llm = ChatOpenAI()

# chain을 생성합니다.
chain = prompt | llm

chain을 `invoke()`하여 실행할 때는 입력 데이터의 타입이 딕셔너리여야 합니다.

In [4]:
# chain을 실행합니다.
chain.invoke({"num" : 5})

AIMessage(content='50', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 15, 'total_tokens': 16, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CqJMt50gCIVVQnCqaXxE7T4zRpikY', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--61f54aaf-96c2-42d8-a1e5-3d4c665045d5-0', usage_metadata={'input_tokens': 15, 'output_tokens': 1, 'total_tokens': 16, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

하지만, langchain 라이브러리가 업데이트 되면서 1개의 변수만 템플릿에 포함되고 있다면, 값만 전달하는 것도 가능합니다.

In [5]:
chain.invoke(5)

AIMessage(content='50', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 15, 'total_tokens': 16, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CqJNhZpFWwqj4Gpxj2C4yGdkCqWdG', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--d3fa3797-18d7-466b-9b41-b7ce86ee73e8-0', usage_metadata={'input_tokens': 15, 'output_tokens': 1, 'total_tokens': 16, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

아래는 `RunnablePassthrough`를 사용한 예제입니다. </br>

`RunnablePassthrough`는 `runnable`객체이며, `runnable`객체는 `invoke()`메서드를 사용하여 별도 실행이 가능합니다.

In [6]:
from langchain_core.runnables import RunnablePassthrough

# runnable
RunnablePassthrough().invoke({"num" : 10})

{'num': 10}

아래는 `RunnablePassthrough`로 체인을 구성하는 예제입니다.

In [7]:
runnable_chain = {"num" : RunnablePassthrough()} | prompt | ChatOpenAI()

# dict 값이 RunnablePassthrough()로 변경되었습니다.
runnable_chain.invoke(10)

AIMessage(content='100', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 15, 'total_tokens': 16, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CqJQqtJhcza2LRzKntoIOQguj9Muu', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--69537184-662a-4bbd-8ef0-ab9a0bdcfad5-0', usage_metadata={'input_tokens': 15, 'output_tokens': 1, 'total_tokens': 16, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

다음은 `RunnablePassthrough.assign()`을 사용하는 경우와 비교한 결과입니다.

In [8]:
RunnablePassthrough().invoke({"num" : 10})

{'num': 10}

`RunnablePassthrough.assign()`
- 입력 값으로 들어온 값의 key/value 쌍과 새롭게 할당된 key/value 쌍을 합칩니다.

In [9]:
# 입력 키 : num, 할당(assign) 키 : new_num
(RunnablePassthrough.assign(new_num=lambda x: x["num"] * 3)).invoke({"num" : 1})

{'num': 1, 'new_num': 3}